In [ ]:
#i'm too lazy to switch keyboards, so it's all in english, sorry
#here we import the modules we'll need to read, count, search and clean tweets
import json
from collections import Counter
import re
import string

def get_file():
    '''puts tweets into a python list line by line'''
    twitter = []
    for line in open('hw_3_twitter.json'):
        twitter.append(json.loads(line))
    return twitter

def get_ids(elem):
    '''gets the user's id'''
    if 'id' in elem: #if it's still there
        right = elem['user']['id']
    elif 'status' in elem['delete']: #if it's gone
        right = elem['delete']['status']['user_id']
    return right

def clean_tags(hashtags):
    '''gets tags - they lie deeper'''
    true_tags = []
    for tag in hashtags:
        for each in tag:
            if 'text' in each:
                true_tags.append(each['text'])
    return true_tags

def clean_tweets(tweet):
    '''here we clean our tweets and lowercase them'''
    tweet_low = tweet.lower()
    tweet_final = tweet_low.translate(str.maketrans('', '', string.punctuation))
    return tweet_final

def main():
    '''does all work'''
    c = Counter()
    twitter = get_file()
    deleted_comms = 0
    languages = []
    orig_ids = []
    repeated_ids = []
    repetitions_count = 0
    followers = {}
    hashtags = []
    apps = []
    noretweets = []
    final_tweets = []
    better_list_for_repeats = []
    for i in twitter: #analysis starts there - with getting id's
        if get_ids(i) in orig_ids or get_ids(i) in repeated_ids:
            repeated_ids.append(get_ids(i))
            repetitions_count += 2
        else:
            orig_ids.append(get_ids(i))
        if 'delete' in i: #work with deleted
            deleted_comms += 1
        else: #work with still existing
            if 'source' in i: #detect source
                app_used = re.search('>.+<', i['source']).group()
                apps.append(app_used.strip('<>'))
            if 'lang' in i: #detect language
                languages.append(i['lang'])
            if 'user' in i: #detect amount of followers
                followers[i['user']['name']] = i['user']['followers_count']
            if 'entities' in i: #detect hachtags
                if not i['entities']['hashtags'] == []:
                    hashtags.append(i['entities']['hashtags'])
            if 'retweeted_status' not in i: #detect retweets and get rid of them
                if 'extended_tweet' in i:
                    noretweets.append(i['extended_tweet']['full_text'])
                else:
                    noretweets.append(i['text'])
    print('\n there are', len(twitter), 'tweets \n')
    percent_deleted = deleted_comms/len(twitter)
    print(percent_deleted*100, '% of them are deleted \n')
    print('the 15 most common languages are: ', Counter(languages).most_common(15), '\n')
    for i in repeated_ids:
        if i not in better_list_for_repeats:
            better_list_for_repeats.append(i)
    print(len(better_list_for_repeats)), 'people have written more than one tweet \n')
    true_tags = clean_tags(hashtags)
    print('the 20 most popular tags are:', Counter(true_tags).most_common(20), '\n')
    for notretweet in noretweets: #clean tweets and give them to Counter
        clean_tweets(notretweet)
        final_tweets.append(notretweet)
    for each in final_tweets:
        c += Counter(each.split())
    print('top-10 lemmas: ', Counter(c).most_common(10), '\n')
    f_list = list(followers.items())
    f_list.sort(key=lambda i: i[1])
    print('these 10 users have the biggest number of followers:', f_list[-10:], '\n')
    true_tags = clean_tags(hashtags)
    print('the 10 most common apps are: ', Counter(apps).most_common(10), '\n')

if __name__ == '__main__':
    main()
